In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
df = pd.read_csv("Advertising.csv")

In [29]:
# Create X and y variables
X = df.drop('sales', axis = 1)
y = df['sales']

# Train | Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



<h5> Create model </h5>

In [30]:
from sklearn.linear_model import ElasticNet

In [31]:
help(ElasticNet)

Help on class ElasticNet in module sklearn.linear_model._coordinate_descent:

class ElasticNet(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, sklearn.linear_model._base.LinearModel)
 |  ElasticNet(alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
 |  
 |  Linear regression with combined L1 and L2 priors as regularizer.
 |  
 |  Minimizes the objective function::
 |  
 |          1 / (2 * n_samples) * ||y - Xw||^2_2
 |          + alpha * l1_ratio * ||w||_1
 |          + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2
 |  
 |  If you are interested in controlling the L1 and L2 penalty
 |  separately, keep in mind that this is equivalent to::
 |  
 |          a * ||w||_1 + 0.5 * b * ||w||_2^2
 |  
 |  where::
 |  
 |          alpha = a + b and l1_ratio = a / (a + b)
 |  
 |  The parameter l1_ratio corresponds to alpha in the glmnet R packa

In [32]:
base_elastic_model = ElasticNet()

<h5>Note that we are trying to explore a method of picking the optimal hyper parameters (alpha, l1_ratio) using a grid rather than using the CV finder that comes built in with ElasticNet. This is why we don't import ElasticNetCV </h5>

## Grid Search

A search consists of:

* an estimator (regressor or classifier such as sklearn.svm.SVC());
* a parameter space;
* a method for searching or sampling candidates;
* a cross-validation scheme 
* a score function.

In [33]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
# 'verbose' parameter specifies how much info we get back

grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=1)

In [36]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [37]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

In [38]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [39]:
pd.DataFrame(grid_model.cv_results_).sort_values('rank_test_score')

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
6        0.000394      0.000126         0.000146        0.000017         0.1   
5        0.000335      0.000011         0.000136        0.000004         0.1   
4        0.000327      0.000007         0.000136        0.000003         0.1   
3        0.000354      0.000038         0.000142        0.000006         0.1   
2        0.000347      0.000017         0.000142        0.000004         0.1   
1        0.000463      0.000043         0.000200        0.000073         0.1   
0        0.000839      0.000170         0.000237        0.000038         0.1   
13       0.000315      0.000005         0.000131        0.000002           1   
12       0.000329      0.000014         0.000133        0.000006           1   
11       0.000330      0.000018         0.000131        0.000002           1   
10       0.000323      0.000009         0.000134        0.000005           1   
9        0.000359      0.000043         0.000135        0.000004           1   
8        0.000340      0.000017         0.000136        0.000002           1   
7        0.000378      0.000041         0.000149        0.000016           1   
14       0.000353      0.000022         0.000136        0.000003           5   
15       0.000389      0.000047         0.000151        0.000009           5   
21       0.000319      0.000005         0.000133        0.000005          10   
16       0.000485      0.000106         0.000162        0.000017           5   
39       0.000315      0.000006         0.000129        0.000001         100   
38       0.000317      0.000005         0.000131        0.000004         100   
37       0.000320      0.000007         0.000135        0.000006         100   
36       0.000315      0.000004         0.000130        0.000002         100   
35       0.000344      0.000028         0.000148        0.000026         100   
34       0.000320      0.000006         0.000132        0.000001          50   
33       0.000321      0.000003         0.000133        0.000004          50   
32       0.000375      0.000087         0.000181        0.000080          50   
31       0.000322      0.000015         0.000133        0.000005          50   
30       0.000314      0.000002         0.000133        0.000005          50   
29       0.000336      0.000013         0.000134        0.000003          50   
20       0.000377      0.000031         0.000148        0.000011           5   
27       0.000334      0.000018         0.000135        0.000006          10   
26       0.000320      0.000009         0.000132        0.000004          10   
25       0.000359      0.000016         0.000141        0.000004          10   
24       0.000343      0.000037         0.000142        0.000018          10   
23       0.000315      0.000005         0.000130        0.000003          10   
22       0.000318      0.000012         0.000135        0.000006          10   
40       0.000322      0.000019         0.000141        0.000011         100   
19       0.000363      0.000036         0.000173        0.000063           5   
18       0.000361      0.000016         0.000149        0.000012           5   
17       0.000399      0.000055         0.000149        0.000009           5   
28       0.000342      0.000023         0.000137        0.000005          50   
41       0.000318      0.000010         0.000133        0.000006         100   

   param_l1_ratio                            params  split0_test_score  \
6               1     {'alpha': 0.1, 'l1_ratio': 1}          -3.206943   
5            0.99  {'alpha': 0.1, 'l1_ratio': 0.99}          -3.208124   
4            0.95  {'alpha': 0.1, 'l1_ratio': 0.95}          -3.213052   
3             0.9   {'alpha': 0.1, 'l1_ratio': 0.9}          -3.221397   
2             0.7   {'alpha': 0.1, 'l1_ratio': 0.7}          -3.269880   
1             0.5   {'alpha': 0.1, 'l1_ratio': 0.5}          -3.325440   
0             0.1   {'alpha': 0.1, 'l1_ratio'

<h5> Use best model from grid search for prediction </h5>

In [40]:
y_pred = grid_model.predict(X_test)

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
MSE = mean_squared_error (y_pred, y_test)

In [43]:
MSE

2.3873426420874737